In [10]:
# Near field
# 
import meep as mp
import numpy as np

# --- Simulation parameters ---
wavelength = 325e-3 # μm
frequency = 1 / wavelength  # Meep units: c = 1 # μm
resolution = 10  # pixels/μm

pml_thickness = 1.0  # μm
padding = 2.0        # distance from source to PML
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

# Define direction vector (unit normalized)
d = np.array([1, 2, 3]) / np.sqrt(14)

# Source properties
source_center = mp.Vector3(0, 0, 0)

sources = [
    mp.Source(mp.ContinuousSource(frequency=frequency),
              component=mp.Ex,
              center=source_center,
              amplitude=d[0]),
    mp.Source(mp.ContinuousSource(frequency=frequency),
              component=mp.Ey,
              center=source_center,
              amplitude=d[1]),
    mp.Source(mp.ContinuousSource(frequency=frequency),
              component=mp.Ez,
              center=source_center,
              amplitude=d[2])
]


def simulate_full_fields(obs_points):
    if obs_points.ndim == 1:
        obs_points = obs_points[np.newaxis, :]

    sim = mp.Simulation(
    cell_size=cell_size,
    resolution=resolution,
    boundary_layers=[mp.PML(pml_thickness)],
    sources=sources  # ✅ Pass the list directly!
    )


    dft_obj = sim.add_dft_fields(
        [mp.Ex, mp.Ey, mp.Ez, mp.Hx, mp.Hy, mp.Hz],
        [frequency],
        center=mp.Vector3(),
        size=cell_size
    )

    optical_cycles = 5
    sim.run(until=optical_cycles * wavelength)



    Ex = sim.get_dft_array(dft_obj, mp.Ex, 0)
    Ey = sim.get_dft_array(dft_obj, mp.Ey, 0)
    Ez = sim.get_dft_array(dft_obj, mp.Ez, 0)
    Hx = sim.get_dft_array(dft_obj, mp.Hx, 0)
    Hy = sim.get_dft_array(dft_obj, mp.Hy, 0)
    Hz = sim.get_dft_array(dft_obj, mp.Hz, 0)

    grid_size = sim.get_array_metadata()  # ✅ this returns (nx, ny, nz)

    delta = 1.0 / resolution
    origin = -0.5 * np.array([cell_dim, cell_dim, cell_dim])

    def to_index(vec):
        return tuple(((vec - origin) / delta).astype(int))

    E_all = []
    H_all = []

    for pt in obs_points:
        idx = to_index(np.array(pt))
        E = np.array([Ex[idx], Ey[idx], Ez[idx]])
        H = np.array([Hx[idx], Hy[idx], Hz[idx]])
        E_all.append(E)
        H_all.append(H)

    return np.array(E_all), np.array(H_all)

# --- Example usage ---
points = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])

points = np.array([[3*wavelength,2*wavelength,4*wavelength],
                  [-2*wavelength,3*wavelength,1*wavelength],
                  [-1*wavelength,-1*wavelength,-1*wavelength]]
)


E, H = simulate_full_fields(points)

for i, pt in enumerate(points):
    print(f"\nPoint {i} at {pt}:")
    print(f"  E = {E[i]}")
    print(f" |E| = {np.linalg.norm(E[i])}")
    print(f"  H = {H[i]}")
    print(f" |H| = {np.linalg.norm(H[i])}")
    print(f" |E|/|H| = {np.linalg.norm(E[i]) / np.linalg.norm(H[i])}")



-----------
Initializing structure...
time for choose_chunkdivision = 0.000288963 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.331545 s
-----------
run 0 finished at t = 1.6500000000000001 (33 timesteps)

Point 0 at [0.975 0.65  1.3  ]:
  E = [ 1.65635794e-04-3.81359358e-05j -8.96864303e-05-1.44812217e-05j
 -2.06231090e-05+4.42142418e-05j]
 |E| = 0.00019880427339546625
  H = [ 1.27352274e-05+1.16345356e-05j  1.30786430e-04-5.98831045e-05j
 -9.14360294e-05+3.60438911e-05j]
 |H| = 0.00017506665755630724
 |E|/|H| = 1.135591872093201

Point 1 at [-0.65   0.975  0.325]:
  E = [0.02591331+0.00626962j 0.00701817-0.00170173j 0.03034692+0.01041948j]
 |E| = 0.04233745050055707
  H = [ 0.01881968+0.01426891j  0.02610367+0.00321835j -0.02367567-0.00690187j]
 |H| = 0.04310121338981316
 |E|/|H| = 0.9822797821873709

Point 2 at [-0.325 -0.325 -0.325]:
  E = [-0.06640763-0.17072433j  0.05538278-0.03696031j  0.17717318+0.09680371j]
 |E| = 0.280

In [7]:
# Far field

import meep as mp
import numpy as np

def simulate_far_field(far_point):
    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=sources
    )

    # Define a near-field box around the source
    nearfield_box = sim.add_near2far(
        frequency, 0, 1,  # fcen, df, nfreq
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(cell_dim - 2 * pml_thickness, 0, 0), direction=mp.X),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, cell_dim - 2 * pml_thickness, 0), direction=mp.Y),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, 0, cell_dim - 2 * pml_thickness), direction=mp.Z)
    )

    sim.run(until=100)

    # Calculate far-field E and H
    ff_fields = sim.get_farfield(nearfield_box, mp.Vector3(*far_point))
    E = np.array(ff_fields[0:3])
    H = np.array(ff_fields[3:6])

    return E, H

# --- Example usage ---
far_point = [-1, 2, 3]
E_far, H_far = simulate_far_field(far_point)

print(f"\nFar-field at {far_point} μm:")
print(f"  E = {E_far}")
print(f" |E| = {np.linalg.norm(E_far)}")
print(f"  H = {H_far}")
print(f" |H| = {np.linalg.norm(H_far)}")
print(f" |E|/|H| = {np.linalg.norm(E_far) / np.linalg.norm(H_far)} (should ≈ 376.73 in vacuum)")
print("E and H are not correctly scaled")


-----------
Initializing structure...
time for choose_chunkdivision = 0.00574613 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.343308 s
-----------
Meep progress: 19.150000000000002/100.0 = 19.2% done in 4.0s, 16.9s to go
on time step 383 (time=19.15), 0.0104612 s/step
Meep progress: 40.75/100.0 = 40.8% done in 8.0s, 11.6s to go
on time step 816 (time=40.8), 0.00925573 s/step
Meep progress: 63.7/100.0 = 63.7% done in 12.0s, 6.8s to go
on time step 1274 (time=63.7), 0.00873572 s/step
Meep progress: 87.65/100.0 = 87.7% done in 16.0s, 2.3s to go
on time step 1753 (time=87.65), 0.00835161 s/step
run 0 finished at t = 100.0 (2000 timesteps)

Far-field at [-1, 2, 3] μm:
  E = [ 3.94060152-6.55677205j  3.1618032 -1.67689043j -0.01335169-1.16604766j]
 |E| = 8.525749613732788
  H = [-1.44297878+0.66475028j  1.18134107-5.11536448j -5.38106476+3.41886742j]
 |H| = 8.410179158356843
 |E|/|H| = 1.013741735247234 (should ≈ 376.73 in vacuum)
E 

In [3]:
E_far

array([ 75.8210864  -1.20650233j,  62.2304824  +2.28936927j,
       -86.27545244+10.33301761j])

In [4]:
# Calculating H using impedence in free space

# Compute radial direction
r_hat = far_point / np.linalg.norm(far_point)


# Impedance of free space
eta0 = 376.730

# Compute far-field H
H_far = np.cross(r_hat, E_far) / eta0

# Display results
print("E_far:", E_far)
print("H_far:", H_far)
print("|E|   =", np.linalg.norm(E_far))
print("|H|   =", np.linalg.norm(H_far))
print("|E|/|H| =", np.linalg.norm(E_far) / np.linalg.norm(H_far))


E_far: [ 75.8210864  -1.20650233j  62.2304824  +2.28936927j
 -86.27545244+10.33301761j]
H_far: [-0.20774969+0.00567272j  0.27707201-0.01765866j  0.01727626+0.00457479j]
|E|   = 131.06636444408767
|H|   = 0.3472639290473506
|E|/|H| = 377.425794851893


In [5]:
import meep as mp
import numpy as np

# --- Simulation Parameters ---
wavelength = 0.325  # μm
frequency = 1 / wavelength
resolution = 10

pml_thickness = 1.0
padding = 2.0
cell_dim = 2 * (padding + pml_thickness)
cell_size = mp.Vector3(cell_dim, cell_dim, cell_dim)

source_pos = mp.Vector3(0, 0, 0)
source_component = mp.Ez

# --- Far-field Simulation Function ---
def simulate_far_field(far_point):
    sim = mp.Simulation(
        cell_size=cell_size,
        resolution=resolution,
        boundary_layers=[mp.PML(pml_thickness)],
        sources=[mp.Source(mp.ContinuousSource(frequency=frequency),
                           component=source_component,
                           center=source_pos)]
    )

    # Near-to-far field monitor surrounding the source
    nearfield_box = sim.add_near2far(
        frequency, 0, 1,  # single frequency
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(cell_dim - 2*pml_thickness, 0, 0), direction=mp.X),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, cell_dim - 2*pml_thickness, 0), direction=mp.Y),
        mp.Near2FarRegion(center=mp.Vector3(), size=mp.Vector3(0, 0, cell_dim - 2*pml_thickness), direction=mp.Z)
    )

    sim.run(until=100)

    r = mp.Vector3(*far_point)
    ff_fields = sim.get_farfield(nearfield_box, r)

    # Split into E and H (6 values returned per frequency)
    ff_E = np.array(ff_fields[:3])
    ff_H = np.array(ff_fields[3:])
    
    return ff_E, ff_H

# --- Main Execution ---
if __name__ == "__main__":
    far_point = [10.0, 0.0, 0.0]  # μm
    eta0 = 376.730  # Vacuum impedance (Ohm)

    E_far, H_meep = simulate_far_field(far_point)
    r_hat = np.array(far_point) / np.linalg.norm(far_point)
    H_calc = np.cross(r_hat, E_far) / eta0

    # --- Print Results ---
    print(f"\n📍 Far-field at {far_point} μm:")
    print("E_far     =", E_far)
    print("H_meep    =", H_meep)
    print("H_calc    =", H_calc)

    print("\n--- Norms ---")
    print(f"|E|        = {np.linalg.norm(E_far):.6f}")
    print(f"|H_meep|   = {np.linalg.norm(H_meep):.6f}")
    print(f"|H_calc|   = {np.linalg.norm(H_calc):.6f}")

    print("\n--- Ratios ---")
    print(f"|E|/|H_meep|  = {np.linalg.norm(E_far)/np.linalg.norm(H_meep):.6f} (should ≈ η = {eta0})")
    print(f"|E|/|H_calc|  = {np.linalg.norm(E_far)/np.linalg.norm(H_calc):.6f} (should = η)")

    print("\n--- Difference ---")
    print(f"ΔH = ||H_meep - H_calc|| = {np.linalg.norm(H_meep - H_calc):.6e}")


-----------
Initializing structure...
time for choose_chunkdivision = 0.001091 s
Working in 3D dimensions.
Computational cell is 6 x 6 x 6 with resolution 10
time for set_epsilon = 0.975995 s
-----------
Meep progress: 7.45/100.0 = 7.5% done in 4.0s, 49.8s to go
on time step 149 (time=7.45), 0.026903 s/step
Meep progress: 20.450000000000003/100.0 = 20.5% done in 8.0s, 31.2s to go
on time step 409 (time=20.45), 0.0154256 s/step
Meep progress: 33.45/100.0 = 33.5% done in 12.0s, 23.9s to go
on time step 670 (time=33.5), 0.0155103 s/step
Meep progress: 43.900000000000006/100.0 = 43.9% done in 16.0s, 20.5s to go
on time step 881 (time=44.05), 0.0189863 s/step
Meep progress: 60.25/100.0 = 60.2% done in 20.1s, 13.2s to go
on time step 1207 (time=60.35), 0.0122956 s/step
Meep progress: 86.2/100.0 = 86.2% done in 24.1s, 3.9s to go
on time step 1727 (time=86.35), 0.00769527 s/step
run 0 finished at t = 100.0 (2000 timesteps)

📍 Far-field at [10.0, 0.0, 0.0] μm:
E_far     = [ 1.35470016e-01+6.866